In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path('').absolute().parents[0]))

import pandas as pd
from scripts.A_import import import_one_file
from scripts.B_cleaning import cleaning
from scripts.C_shaping import *
from scripts.D_output import save

In [3]:
df = import_one_file()
df_original = df.copy()

/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/analisi_comp/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:

df_cleaned = cleaning(df, adding_area=True)
df_label = create_comp_today(df_cleaned, week_inside_first=1)
df_final = create_comp_today_no_outliers(df_label)
#save(df_final)
#save(df_final, name = 'df_comp.parquet')
save(df_final, name = 'df_comp.csv')
#path_output = Path('/Volumes/share/Gruppo_Demand_Planning/02_NPI/DB WHOLESALE NPI/ANALISI COMP NEW')
#df_final.to_excel(path_output / 'df_comp_cleaned.xlsx', index=False)

In [4]:
df2

NameError: name 'df2' is not defined

In [ ]:
df2.to_excel(Path('').absolute().parents[0] / 'data' /'output'/ 'df_comp_cleaned.xlsx', index=False)

In [ ]:
sys.exit()

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import random

brand = 'RX'
customer = random.choice(df2.customer.unique())
release = '2023 N1'
creation_week = 202301

df.query('brand == @brand & customer_sold_to == @customer & release == @release & creation_yearweek == @creation_week')
df2.query('brand == @brand & customer == @customer & release == @release').sort_values(by='creation_yearweek')

In [ ]:
df_outliers_iqr = detect_outlier(df_label)

df_outliers_mad = detect_outlier(df_label, 'mad')

df_outliers_iso = detect_outlier(df_label, 'iso')

#df_outliers_svm = detect_outlier(df_label, 'svm')

In [ ]:
df_outliers_iso

In [ ]:
df_outliers_mad

In [ ]:
dict_outliers = {'iqr': df_outliers_iqr, 'mad': df_outliers_mad, 'iso': df_outliers_iso}

In [ ]:
import plotly.express as px

# Select a specific combination for demonstration (you can adjust these values as needed)
selected_brand = 'RB'
selected_dedalo_area = 'MUESED'
selected_acquisition_mode_group = 'REPLENISHMENT'

METHOD = 'iqr'
df_outliers = dict_outliers[METHOD]

# Filter the DataFrame for the selected combination
filtered_df = df_outliers[(df_outliers['brand'] == selected_brand) &
                        (df_outliers['dedalo_area'] == selected_dedalo_area) &
                        (df_outliers['new_acquisition_mode'] == selected_acquisition_mode_group)]

# Create a scatter plot
fig = px.scatter(filtered_df, x='creation_yearweek', y='qty', color='outlier',
                color_discrete_map={True: 'red', False: 'blue'},
                title=f"Qty Outliers for {selected_brand}, {selected_dedalo_area}, {selected_acquisition_mode_group}")

# Show figure
fig.show()

In [ ]:
df_outliers.query('brand == @selected_brand' )

In [ ]:
sys.exit()

In [ ]:
create_till_today(df_label)

In [ ]:
create_comp_today(df)

In [ ]:
df_label

In [ ]:
df_label.to_csv('/Users/mottad/Library/CloudStorage/OneDrive-LuxotticaGroupS.p.A/Desktop/df_comp.csv', index=False)

In [ ]:
customer = 'OPTICAL CENTER'
brand = 'RX'

df_label.query('customer == @customer & brand == @brand')

un cliente, più aree

In [ ]:
customer = df.groupby(by = 'customer', as_index=False)['dedalo_area'].nunique().query('dedalo_area >1').customer.iloc[1]
df.query('customer == @customer')


In [ ]:
df.acquisition_mode_group.unique()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import random



In [ ]:
brand = 'OX'
dedalo = 'MUESED'
acquisition = 'REP'

plt.figure(1, figsize=(20,10))
sns.histplot(data = df.query('brand == @brand & dedalo_area == @dedalo & acquisition_mode_group == @acquisition & datest_name in ["Turchia", "Italia", "Turchia Sport", "Italia Sport"]'), x = 'qty', hue = 'datest_name')
plt.xlim((0,100))

plt.figure(2)
sns.boxplot(data = df.query('brand == @brand & dedalo_area == @dedalo & acquisition_mode_group == @acquisition'), x = 'qty')

In [ ]:
import plotly.graph_objects as go

# Assuming df is your DataFrame filtered by brand, dedalo_area, and acquisition_mode_group
# and contains columns for qty and datest_name

# Unique datest_names for dropdown options
datest_names = df['datest_name'].unique()

# Create a figure
fig = go.Figure()

for name in datest_names:
    # Filter the DataFrame for each datest_name
    df_filtered = df[df['datest_name'] == name]
    
    # Add a histogram trace for each datest_name
    fig.add_trace(
        go.Histogram(
            x=df_filtered['qty'],
            name=name,  # Use datest_name as the trace name
            opacity=0.75
        )
    )

# Update layout to add a dropdown menu
fig.update_layout(
    updatemenus=[
        dict(
            buttons=[
                dict(
                    label=name,
                    method="update",
                    args=[{"visible": [name == dname for dname in datest_names]},
                          {"title": f"Data for: {name}"}]
                ) for name in datest_names
            ] + [
                dict(
                    label="All",
                    method="update",
                    args=[{"visible": [True] * len(datest_names)},
                          {"title": "All Data"}]
                )
            ],
            direction="down",
            showactive=True,
        )
    ],
    barmode='overlay',
    xaxis=dict(range=[500, 3000]),  # Limit the x-axis to 200
    yaxis=dict(range=[0, 5])  # Limit the x-axis to 200
)

# Optionally, add functionality to hide other traces when one is clicked in the legend
fig.update_traces(selector=dict(type='histogram'), showlegend=True)

fig.show()


In [ ]:
df.query('brand == @brand & dedalo_area == @dedalo & acquisition_mode_group == @acquisition & qty > 400')

In [ ]:
grouped = df.groupby(['brand', 'acquisition_mode_group','dedalo_area'])

# Calculate the quartiles for each group
quartiles = grouped['qty'].quantile([0.1, 0.5, 0.9]).unstack()

In [ ]:
quartiles

In [ ]:
df_prova = df.groupby(by = 'datest_name', as_index=False)['qty'].sum()

plt.figure(figsize = (30,10))
sns.barplot(data = df_prova, x = df_prova['datest_name'], y= df_prova['qty'])
plt.xticks(rotation=45)

In [ ]:
df.query('datest_name == "Turchia" & qty >= 200')

In [ ]:

customer = random.choice(df.customer.unique())
sns.histplot(data=df.query("customer == @customer"), x='planned_delivery_date', hue='release', bins = 20)

In [ ]:
dedalo_area = 'MUESED'
plt.figure(figsize = (20,15))

df3 = df2.groupby(by = [col for col in df2.columns if not col in ['planned_delivery_date', 'qty']], as_index=False)['qty'].sum()

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))

df_label = label_first_order(df3, week_inside_first=1).query('dedalo_area == @dedalo_area')    
sns.histplot(data=df_label, x='acquisition_mode_group', hue='first_order', bins = 20, ax=ax1)

df_label = label_first_order(df3, week_inside_first=2).query('dedalo_area == @dedalo_area')        
sns.histplot(data=df_label, x='acquisition_mode_group', hue='first_order', bins = 20, ax=ax2)

df_label = label_first_order(df3, week_inside_first=4).query('dedalo_area == @dedalo_area')        
sns.histplot(data=df_label, x='acquisition_mode_group', hue='first_order', bins = 20, ax=ax3)

df_label = label_first_order(df3, week_inside_first=6).query('dedalo_area == @dedalo_area')        
sns.histplot(data=df_label, x='acquisition_mode_group', hue='first_order', bins = 20, ax=ax4)





In [ ]:
number_of_week = range(1, 20)
lst_first_order = []
for i in number_of_week:
    lst_first_order.append(len(label_first_order(df_cleaned, week_inside_first=i).query('first_order == True')) / len(df_cleaned))

df_number_first_orders = pd.DataFrame({'number_of_week': number_of_week, 'indicator': lst_first_order})

sns.lineplot(data=df_number_first_orders, x = 'number_of_week', y = 'indicator')

In [ ]:
df_label.query("acquisition_mode_group == 'RED CARPET' & first_order == False & release == '2023 N1'")

In [ ]:
customer = "ECOUTER VOIR LES OPT MUTUALISTES"
brand = 'VO'
amg = 'REP'
rel = '2024 N1'

df.query("customer == @customer & brand == @brand")
df.query("customer == @customer & brand == @brand & release == @rel").sort_values(by = 'creation_yearweek')